In [1]:
from src.lib.utils.FileHandling.FileHandlingInterface import *
from src.lib.SVCDSeg.SVCDSeg import *
import matplotlib.pyplot as plt
import logging
import time
import warnings

In [2]:
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.DEBUG)
warnings.simplefilter("always", RuntimeWarning)

In [3]:
PASCAL_DEMO_IMAGE_PATH = r"src\data\demoPascal\2008_003147.jpg"
PASCAL_DEMO_IMAGE_PATH1 = r"src\data\demoPascal\2010_005258.jpg"

In [4]:
PASCAL_DEMO_SCRIBBLE_PATH = r"src\data\demoPascal\2008_003147.xml"
PASCAL_DEMO_SCRIBBLE_PATH1 = r"src\data\demoPascal\2010_005258.xml"

In [5]:
sample_image = TargetImage(
    PASCAL_DEMO_IMAGE_PATH1              
)

In [6]:
sample_scribble = EncodedScribble(
    PASCAL_DEMO_SCRIBBLE_PATH1
)

In [7]:
n_classes = sample_scribble.get_n_classes()

In [8]:
segmenter = SVCDSeg(
    n_classes,
    debug = 0
)

In [9]:
segmenter.fit(
    sample_image,
    sample_scribble 
)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\lollo\Desktop\GM\T2_DSBA_Graphical_Methods\src\lib\probabilityEstim\Likelihood.py:159: RuntimeWarning: invalid value encountered in divide
  kde_likelihood_map /= np.sum(kde_likelihood_map, axis = 0) # normalize to sum to one over each class
C:\Users\lollo\Desktop\GM\T2_DSBA_Graphical_Methods\src\lib\probabilityEstim\Likelihood.py:162: RuntimeWarning: divide by zero encountered in log
  kde_likelihood = -1 * np.log(kde_likelihood_map)
C:\Users\lollo\Desktop\GM\T2_DSBA_Graphical_Methods\src\lib\SVCDSeg\SVCDSeg.py:101: RuntimeWarning: invalid value encountered in divide
  norm_xi = np.sqrt(xi[0]**2 + xi[1]**2) / fitted_prior
C:\Users\lollo\Desktop\GM\T2_DSBA_Graphical_Methods\src\lib\SVCDSeg\SVCDSeg.py:137: RuntimeWarning: divide by zero encountered in divide
  theta = 1 / p * np.take_along_axis(sum_vecs, indices=pn, axis=0)
  1%|▊                               

 24%|███████████████████▍                                                             | 24/100 [00:08<00:21,  3.47it/s]C:\Users\lollo\Desktop\GM\T2_DSBA_Graphical_Methods\src\lib\SVCDSeg\SVCDSeg.py:137: RuntimeWarning: divide by zero encountered in divide
  theta = 1 / p * np.take_along_axis(sum_vecs, indices=pn, axis=0)
 25%|████████████████████▎                                                            | 25/100 [00:08<00:22,  3.33it/s]C:\Users\lollo\Desktop\GM\T2_DSBA_Graphical_Methods\src\lib\SVCDSeg\SVCDSeg.py:137: RuntimeWarning: divide by zero encountered in divide
  theta = 1 / p * np.take_along_axis(sum_vecs, indices=pn, axis=0)
 26%|█████████████████████                                                            | 26/100 [00:09<00:22,  3.36it/s]C:\Users\lollo\Desktop\GM\T2_DSBA_Graphical_Methods\src\lib\SVCDSeg\SVCDSeg.py:137: RuntimeWarning: divide by zero encountered in divide
  theta = 1 / p * np.take_along_axis(sum_vecs, indices=pn, axis=0)
 27%|█████████████████████▊    

 49%|███████████████████████████████████████▋                                         | 49/100 [00:16<00:15,  3.30it/s]C:\Users\lollo\Desktop\GM\T2_DSBA_Graphical_Methods\src\lib\SVCDSeg\SVCDSeg.py:137: RuntimeWarning: divide by zero encountered in divide
  theta = 1 / p * np.take_along_axis(sum_vecs, indices=pn, axis=0)
 50%|████████████████████████████████████████▌                                        | 50/100 [00:16<00:15,  3.29it/s]C:\Users\lollo\Desktop\GM\T2_DSBA_Graphical_Methods\src\lib\SVCDSeg\SVCDSeg.py:137: RuntimeWarning: divide by zero encountered in divide
  theta = 1 / p * np.take_along_axis(sum_vecs, indices=pn, axis=0)
 51%|█████████████████████████████████████████▎                                       | 51/100 [00:16<00:14,  3.37it/s]C:\Users\lollo\Desktop\GM\T2_DSBA_Graphical_Methods\src\lib\SVCDSeg\SVCDSeg.py:137: RuntimeWarning: divide by zero encountered in divide
  theta = 1 / p * np.take_along_axis(sum_vecs, indices=pn, axis=0)
 52%|██████████████████████████

 74%|███████████████████████████████████████████████████████████▉                     | 74/100 [00:23<00:07,  3.51it/s]C:\Users\lollo\Desktop\GM\T2_DSBA_Graphical_Methods\src\lib\SVCDSeg\SVCDSeg.py:137: RuntimeWarning: divide by zero encountered in divide
  theta = 1 / p * np.take_along_axis(sum_vecs, indices=pn, axis=0)
 75%|████████████████████████████████████████████████████████████▊                    | 75/100 [00:23<00:07,  3.49it/s]C:\Users\lollo\Desktop\GM\T2_DSBA_Graphical_Methods\src\lib\SVCDSeg\SVCDSeg.py:137: RuntimeWarning: divide by zero encountered in divide
  theta = 1 / p * np.take_along_axis(sum_vecs, indices=pn, axis=0)
 76%|█████████████████████████████████████████████████████████████▌                   | 76/100 [00:23<00:07,  3.36it/s]C:\Users\lollo\Desktop\GM\T2_DSBA_Graphical_Methods\src\lib\SVCDSeg\SVCDSeg.py:137: RuntimeWarning: divide by zero encountered in divide
  theta = 1 / p * np.take_along_axis(sum_vecs, indices=pn, axis=0)
 77%|██████████████████████████

 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [00:30<00:00,  3.33it/s]C:\Users\lollo\Desktop\GM\T2_DSBA_Graphical_Methods\src\lib\SVCDSeg\SVCDSeg.py:137: RuntimeWarning: divide by zero encountered in divide
  theta = 1 / p * np.take_along_axis(sum_vecs, indices=pn, axis=0)
100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:30<00:00,  3.23it/s]


array([[[0.3396915 , 0.33968209, 0.33966302, ...,        nan,
                nan,        nan],
        [0.33968849, 0.33967919, 0.3396603 , ...,        nan,
                nan,        nan],
        [0.33968252, 0.33967343, 0.33965486, ...,        nan,
                nan,        nan],
        ...,
        [0.33385087, 0.33385044, 0.33384969, ..., 0.33336428,
         0.33335395, 0.33334364],
        [0.33367811, 0.33367776, 0.33367716, ..., 0.33335395,
         0.33334707, 0.3333402 ],
        [0.33350566, 0.33350547, 0.33350513, ..., 0.33334364,
         0.3333402 , 0.33333676]],

       [[0.3218365 , 0.32185393, 0.3218892 , ...,        nan,
                nan,        nan],
        [0.3218487 , 0.32186632, 0.32190199, ...,        nan,
                nan,        nan],
        [0.32187269, 0.32189068, 0.32192712, ...,        nan,
                nan,        nan],
        ...,
        [0.33268905, 0.33269475, 0.33270545, ..., 0.33335374,
         0.33334874, 0.33334164],
        [0.3